In [2]:
# Project: Does the retail price of gasoline affect US presidential approval?

# Edward Natusch - Data Science - University of Bristol 

# In this notebook, I will scrape presidential Polling Data for Biden, Trump and Obama from Rasmussen Reports using pandas.

# I will then download retail gasoline prices (all grades all formulations) from the Fred API.

# I will then merge data these data sets, creating CSVs with weekly polling averages and the weekly retail price of gasoline for each presidency.

In [3]:
import pandas as pd

Trump_Approval = pd.read_html('https://www.rasmussenreports.com/public_content/politics/Trump_administration/Trump_approval_index_history')

# Loading and cleaning the data 

DF_Trump_Approval = Trump_Approval[0]

DF_Trump_Approval["Date"] = pd.to_datetime(DF_Trump_Approval["Date"])

DF_Trump_Approval['Total Approve'] = DF_Trump_Approval['Total Approve'].str.replace('%',' ')
DF_Trump_Approval['Total Approve'] = pd.to_numeric(DF_Trump_Approval['Total Approve'], errors ='coerce')
DF_Trump_Approval.dropna(subset = ["Total Approve"], inplace=True)

DF_Trump_Approval['Total Disapprove'] = DF_Trump_Approval['Total Disapprove'].str.replace('%',' ')
DF_Trump_Approval['Total Disapprove'] = pd.to_numeric(DF_Trump_Approval['Total Disapprove'], errors ='coerce')
DF_Trump_Approval.dropna(subset = ["Total Disapprove"], inplace=True)

DF_Trump_Approval['Strongly Approve'] = DF_Trump_Approval['Strongly Approve'].str.replace('%',' ')
DF_Trump_Approval['Strongly Approve'] = pd.to_numeric(DF_Trump_Approval['Strongly Approve'], errors ='coerce')
DF_Trump_Approval.dropna(subset = ["Strongly Approve"], inplace=True)


DF_Trump_Approval['Strongly Disapprove'] = DF_Trump_Approval['Strongly Disapprove'].str.replace('%',' ')
DF_Trump_Approval['Strongly Disapprove'] = pd.to_numeric(DF_Trump_Approval['Strongly Disapprove'], errors ='coerce')
DF_Trump_Approval.dropna(subset = ["Strongly Disapprove"], inplace=True)

# Creating Weekly averages for presidential approval

DF_Trump_Approval_Weekly = DF_Trump_Approval.resample('W', on='Date').mean().reset_index()

# off-setting the day by 1 so the weekly average of the previous week is reflected on the monday of the next week.
# (This is so the data can be merged with the FRED data, which has a weekly frequency ending Monday)

DF_Trump_Approval_Weekly['Date'] = pd.DatetimeIndex(DF_Trump_Approval_Weekly['Date']) + pd.DateOffset(1)

In [4]:
# Importing the required packages to download the data from the API.

import requests

import os 

import json

from google.colab import files

In [5]:
# Downloading the Json file from th Fred API and loading it into a pandas data frame 

url = 'https://api.stlouisfed.org/fred/series/observations?series_id=GASALLW&api_key=e725511ce5615fe46441618aca9c09bf&file_type=json'

data = requests.get(url)

Retail_Gasoline_Consumer_Price = data.json()

Observations = Retail_Gasoline_Consumer_Price['observations']

Retail_Gasoline_Consumer_Price_Array=[]

for x in Observations:
  temp_variable={'date':x['date'],'value':x['value']}
  Retail_Gasoline_Consumer_Price_Array.append(temp_variable)

DF_Retail_Gasoline_Price  = pd.DataFrame(Retail_Gasoline_Consumer_Price_Array)

# Adding New Column Names 

DF_Retail_Gasoline_Price.columns = ['Date','Retail Price of Gasoline (Dollars per Gallon)']

# Formatting Dates

DF_Retail_Gasoline_Price['Date'] = pd.to_datetime(DF_Retail_Gasoline_Price['Date'])

# Formatting Price Index

DF_Retail_Gasoline_Price['Retail Price of Gasoline (Dollars per Gallon)'] = pd.to_numeric(DF_Retail_Gasoline_Price['Retail Price of Gasoline (Dollars per Gallon)'], errors ='coerce')


In [6]:
# Merging the data from the two data sources to create a single data set 

DF_Trump_Approval_Weekly_With_Gasoline_Prices = DF_Trump_Approval_Weekly.merge(DF_Retail_Gasoline_Price, left_on='Date', right_on='Date')

# Creating a new year field, so a slider in vega lite based on the year field can be produced

DF_Trump_Approval_Weekly_With_Gasoline_Prices['Year'] = pd.DatetimeIndex(DF_Trump_Approval_Weekly_With_Gasoline_Prices['Date']).year

DF_Trump_Approval_Weekly_With_Gasoline_Prices.to_csv("Project_President_Trump_Polling_Weekly_Retail_Price_of_Gasoline.csv")

In [7]:
# Creating the same data set for Barack Obama

import pandas as pd

Obama_Approval = pd.read_html('https://www.rasmussenreports.com/public_content/politics/obama_administration/obama_approval_index_history')

# Loading and cleaning the data 

DF_Obama_Approval = Obama_Approval[0]

DF_Obama_Approval["Date"] = pd.to_datetime(DF_Obama_Approval["Date"])

DF_Obama_Approval['Total Approve'] = DF_Obama_Approval['Total Approve'].str.replace('%',' ')
DF_Obama_Approval['Total Approve'] = pd.to_numeric(DF_Obama_Approval['Total Approve'], errors ='coerce')
DF_Obama_Approval.dropna(subset = ["Total Approve"], inplace=True)

DF_Obama_Approval['Total Disapprove'] = DF_Obama_Approval['Total Disapprove'].str.replace('%',' ')
DF_Obama_Approval['Total Disapprove'] = pd.to_numeric(DF_Obama_Approval['Total Disapprove'], errors ='coerce')
DF_Obama_Approval.dropna(subset = ["Total Disapprove"], inplace=True)

DF_Obama_Approval['Strongly Approve'] = DF_Obama_Approval['Strongly Approve'].str.replace('%',' ')
DF_Obama_Approval['Strongly Approve'] = pd.to_numeric(DF_Obama_Approval['Strongly Approve'], errors ='coerce')
DF_Obama_Approval.dropna(subset = ["Strongly Approve"], inplace=True)


DF_Obama_Approval['Strongly Disapprove'] = DF_Obama_Approval['Strongly Disapprove'].str.replace('%',' ')
DF_Obama_Approval['Strongly Disapprove'] = pd.to_numeric(DF_Obama_Approval['Strongly Disapprove'], errors ='coerce')
DF_Obama_Approval.dropna(subset = ["Strongly Disapprove"], inplace=True)

# Creating Weekly averages for presidential approval

DF_Obama_Approval_Weekly = DF_Obama_Approval.resample('W', on='Date').mean().reset_index()

# off-setting the day by 1 so the weekly average of the previous week is reflected on the monday of the next week.
# (This is so the data can be merged with the FRED data, which has a weekly frequency ending Monday)

DF_Obama_Approval_Weekly['Date'] = pd.DatetimeIndex(DF_Obama_Approval_Weekly['Date']) + pd.DateOffset(1)

In [8]:
# Merging the data from the two data sources to create a single data set 

DF_Obama_Approval_Weekly_With_Gasoline_Prices = DF_Obama_Approval_Weekly.merge(DF_Retail_Gasoline_Price, left_on='Date', right_on='Date')

# Creating a new year field, so a slider in vega lite based on the year field can be produced

DF_Obama_Approval_Weekly_With_Gasoline_Prices['Year'] = pd.DatetimeIndex(DF_Obama_Approval_Weekly_With_Gasoline_Prices['Date']).year

DF_Obama_Approval_Weekly_With_Gasoline_Prices.to_csv("Project_President_Obama_Polling_Weekly_Retail_Price_of_Gasoline.csv")

In [9]:
# Creating the same data set for President biden

Biden_Approval = pd.read_html('https://www.rasmussenreports.com/public_content/politics/Biden_administration/Biden_approval_index_history')

# Loading and cleaning the data 

DF_Biden_Approval = Biden_Approval[0]

DF_Biden_Approval["Date"] = pd.to_datetime(DF_Biden_Approval["Date"])

DF_Biden_Approval['Total Approve'] = DF_Biden_Approval['Total Approve'].str.replace('%',' ')
DF_Biden_Approval['Total Approve'] = pd.to_numeric(DF_Biden_Approval['Total Approve'], errors ='coerce')
DF_Biden_Approval.dropna(subset = ["Total Approve"], inplace=True)

DF_Biden_Approval['Total Disapprove'] = DF_Biden_Approval['Total Disapprove'].str.replace('%',' ')
DF_Biden_Approval['Total Disapprove'] = pd.to_numeric(DF_Biden_Approval['Total Disapprove'], errors ='coerce')
DF_Biden_Approval.dropna(subset = ["Total Disapprove"], inplace=True)

DF_Biden_Approval['Strongly Approve'] = DF_Biden_Approval['Strongly Approve'].str.replace('%',' ')
DF_Biden_Approval['Strongly Approve'] = pd.to_numeric(DF_Biden_Approval['Strongly Approve'], errors ='coerce')
DF_Biden_Approval.dropna(subset = ["Strongly Approve"], inplace=True)


DF_Biden_Approval['Strongly Disapprove'] = DF_Biden_Approval['Strongly Disapprove'].str.replace('%',' ')
DF_Biden_Approval['Strongly Disapprove'] = pd.to_numeric(DF_Biden_Approval['Strongly Disapprove'], errors ='coerce')
DF_Biden_Approval.dropna(subset = ["Strongly Disapprove"], inplace=True)

# Creating Weekly averages for presidential approval

DF_Biden_Approval_Weekly = DF_Biden_Approval.resample('W', on='Date').mean().reset_index()

# off-setting the day by 1 so the weekly average of the previous week is reflected on the monday of the next week.
# (This is so the data can be merged with the FRED data, which has a weekly frequency ending Monday)

DF_Biden_Approval_Weekly['Date'] = pd.DatetimeIndex(DF_Biden_Approval_Weekly['Date']) + pd.DateOffset(1)

In [10]:
# Merging the data from the two data sources to create a single data set 

DF_Biden_Approval_Weekly_With_Gasoline_Prices = DF_Biden_Approval_Weekly.merge(DF_Retail_Gasoline_Price, left_on='Date', right_on='Date')

# Creating a new year field, so a slider in vega lite based on the month field can be produced

DF_Biden_Approval_Weekly_With_Gasoline_Prices['Month'] = pd.DatetimeIndex(DF_Biden_Approval_Weekly_With_Gasoline_Prices['Date']).month

DF_Biden_Approval_Weekly_With_Gasoline_Prices.to_csv("Project_President_Biden_Polling_Weekly_Retail_Price_of_Gasoline.csv")